# Document Retrieval using TF-IDF Weighted Rank and TF-IDF Cosine Similarity

## Imports

In [1]:
# !unzip stories

In [1]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words

import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math

# %load_ext autotime

In [2]:
title = "stories"
alpha = 0.3

## Taking all folders

In [3]:
folders = [x[0] for x in os.walk(str(os.getcwd())+'/'+title+'/')]
folders[0] = folders[0][:len(folders[0])-1]

In [4]:
folders

['/home/nutsa/Desktop/search_pdfs/stories',
 '/home/nutsa/Desktop/search_pdfs/stories/FARNON',
 '/home/nutsa/Desktop/search_pdfs/stories/SRE']

## Collecting the file names and titles

In [5]:
dataset = []

c = False

for i in folders:
    file = open(i+"/index.html", 'r')
    text = file.read().strip()
    file.close()

    file_name = re.findall('><A HREF="(.*)">', text)
    file_title = re.findall('<BR><TD> (.*)\n', text)

    if c == False:
        file_name = file_name[2:]
        c = True
        
    print(len(file_name), len(file_title))

    for j in range(len(file_name)):
        dataset.append((str(i) +"/"+ str(file_name[j]), file_title[j]))

452 452
0 0
15 15


In [6]:
len(dataset)

467

In [7]:
dataset

[('/home/nutsa/Desktop/search_pdfs/stories/100west.txt',
  'Going 100 West by 53 North by Jim Prentice (1990)'),
 ('/home/nutsa/Desktop/search_pdfs/stories/13chil.txt',
  'The Story of the Sly Fox'),
 ('/home/nutsa/Desktop/search_pdfs/stories/14.lws',
  'A Smart Bomb with a Language Parser'),
 ('/home/nutsa/Desktop/search_pdfs/stories/16.lws',
  'Two Guys in a Garage, by M. Pshota'),
 ('/home/nutsa/Desktop/search_pdfs/stories/17.lws',
  'The Early Days of a High-Tech Start-up are Magic (November 18, 1991) by M. Peshota'),
 ('/home/nutsa/Desktop/search_pdfs/stories/18.lws',
  'The Couch, the File Cabinet, and the Calendar, by M. Peshota (December 9, 1991)'),
 ('/home/nutsa/Desktop/search_pdfs/stories/19.lws',
  'Engineering the Future of American Technology by M. Peshota (January 5, 1992)'),
 ('/home/nutsa/Desktop/search_pdfs/stories/20.lws',
  'What Research and Development Was Always Meant to Be, by M. Peshota'),
 ('/home/nutsa/Desktop/search_pdfs/stories/3gables.txt',
  'The Adventur

In [8]:
N = len (dataset)

In [9]:
def print_doc(id):
    print(dataset[id])
    file = open(dataset[id][0], 'r', encoding='cp1250')
    text = file.read().strip()
    file.close()
    print(text)

# Preprocessing

In [10]:
def convert_lower_case(data):
    return np.char.lower(data)

In [11]:
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

In [12]:
def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

In [13]:
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

In [14]:
def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

In [16]:
def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

In [17]:
def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = convert_numbers(data)
    data = stemming(data)
    data = remove_punctuation(data)
    data = convert_numbers(data)
    data = stemming(data) #needed again as we need to stem the words
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    return data

In [20]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/nutsa/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [22]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/nutsa/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Extracting Data

In [18]:
processed_text = []
processed_title = []

for i in dataset[:N]:
    file = open(i[0], 'r', encoding="utf8", errors='ignore')
    text = file.read().strip()
    file.close()

    processed_text.append(word_tokenize(str(preprocess(text))))
    processed_title.append(word_tokenize(str(preprocess(i[1]))))

## Calculating DF for all words

In [19]:
DF = {}

for i in range(N):
    tokens = processed_text[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}

    tokens = processed_title[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}
for i in DF:
    DF[i] = len(DF[i])

In [19]:
# DF

In [20]:
total_vocab_size = len(DF)

In [21]:
total_vocab_size

32350

In [22]:
total_vocab = [x for x in DF]

In [23]:
print(total_vocab[:20])

['sharewar', 'trial', 'project', 'freewar', 'need', 'support', 'continu', 'one', 'hundr', 'west', 'fifti', 'three', 'north', 'jim', 'prentic', 'copyright', 'thousand', 'nine', 'nineti', 'brandon']


In [24]:
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

### Calculating TF-IDF for body, we will consider this as the actual tf-idf as we will add the title weight to this.

In [25]:
doc = 0

tf_idf = {}

for i in range(N):
    
    tokens = processed_text[i]
    
    counter = Counter(tokens + processed_title[i])
    words_count = len(tokens + processed_title[i])
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))
        
        tf_idf[doc, token] = tf*idf

    doc += 1

In [26]:
# tf_idf

### Calculating TF-IDF for Title

In [26]:
doc = 0

tf_idf_title = {}

for i in range(N):
    
    tokens = processed_title[i]
    counter = Counter(tokens + processed_text[i])
    words_count = len(tokens + processed_text[i])

    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1)) #numerator is added 1 to avoid negative values
        
        tf_idf_title[doc, token] = tf*idf

    doc += 1

In [28]:
# tf_idf_title

In [27]:
tf_idf[(0,"go")]

0.0002906893990853149

In [28]:
tf_idf_title[(0,"go")]

0.0002906893990853149

## Merging the TF-IDF according to weights

In [29]:
for i in tf_idf:
    tf_idf[i] *= alpha

In [30]:
for i in tf_idf_title:
    tf_idf[i] = tf_idf_title[i]

In [31]:
len(tf_idf)

344378

# TF-IDF Matching Score Ranking

In [32]:
def matching_score(k, query):
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))

    print("Matching Score")
    print("\nQuery:", query)
    print("")
    print(tokens)
    
    query_weights = {}

    for key in tf_idf:
        
        if key[1] in tokens:
            try:
                query_weights[key[0]] += tf_idf[key]
            except:
                query_weights[key[0]] = tf_idf[key]
    
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)

    print("")
    
    l = []
    
    for i in query_weights[:10]:
        l.append(i[0])
    
    print(l)
    

matching_score(10, "Without the drive of Rebeccah's insistence, Kate lost her momentum. She stood next a slatted oak bench, canisters still clutched, surveying")

Matching Score

Query: Without the drive of Rebeccah's insistence, Kate lost her momentum. She stood next a slatted oak bench, canisters still clutched, surveying

['without', 'drive', 'rebeccah', 'insist', 'kate', 'lost', 'momentum', 'stood', 'next', 'slat', 'oak', 'bench', 'canist', 'still', 'clutch', 'survey']

[166, 200, 352, 433, 211, 350, 175, 187, 188, 294]


In [35]:
# print_doc(2)

# TF-IDF Cosine Similarity Ranking

In [33]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

### Vectorising tf-idf

In [34]:
D = np.zeros((N, total_vocab_size))
for i in tf_idf:
    try:
        ind = total_vocab.index(i[1])
        D[i[0]][ind] = tf_idf[i]
    except:
        pass

In [35]:
def gen_vector(tokens):

    Q = np.zeros((len(total_vocab)))
    
    counter = Counter(tokens)
    words_count = len(tokens)

    query_weights = {}
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = math.log((N+1)/(df+1))

        try:
            ind = total_vocab.index(token)
            Q[ind] = tf*idf
        except:
            pass
    return Q

In [60]:
def cosine_similarity(k, query):
    print("Cosine Similarity")
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))
    
    print("\nQuery:", query)
    print("")
    print(tokens)
    
    d_cosines = []
    
    query_vector = gen_vector(tokens)
    
    for d in D:
        d_cosines.append(cosine_sim(query_vector, d))
        
    out = np.array(d_cosines).argsort()[-k:][::-1]
    
    print("")
    
    print(out)
    
    print("")
    
    print_doc_custom(out)

#     for i in out:
#         print(i, dataset[i][0])

Q = cosine_similarity(10, "Without the drive of Rebeccah's insistence, Kate lost her momentum. She stood next a slatted oak bench, canisters still clutched, surveying")

Cosine Similarity

Query: Without the drive of Rebeccah's insistence, Kate lost her momentum. She stood next a slatted oak bench, canisters still clutched, surveying

['without', 'drive', 'rebeccah', 'insist', 'kate', 'lost', 'momentum', 'stood', 'next', 'slat', 'oak', 'bench', 'canist', 'still', 'clutch', 'survey']

[200 166 433 175 169 402 211  87 151 369]

Time for Flowers, by Gay Bost

Survey Results, by Joe DeRouen (1994)

The Valentine's Day Blues, by B.A. Nilsson

A Chance Meeting in the Park, by Joe DeRouen

A List of Father Good Stories

The Story of SUPERGUY!!

Nightmare in Grey by Fredric Brown

They Don't Come Any Larger: The Continuing Adventures of Mr. X.

Encoded in Strands, by Gay Bost

In Residence by KaRylin



In [61]:
Q = cosine_similarity(10, "Beautiful flowers")

Cosine Similarity

Query: Beautiful flowers

['beauti', 'flower']

[200  89  49  84 140 293 355 279 418  93]

Time for Flowers, by Gay Bost

No Options Left, by G. Daniel Flower

The Story of Beauty and the Beast

The Hunter, the Hunted, a story by G. Daniel Flower

The Domain of Arnheim

Poem: Through My Eyes

The Rose, by Suzy Edmonson

A Day in McDonaldland by G.I. Joe (June 10, 1986)

The Time Machine, by H.G. Wells (1898)

Prints of the City, by G. Daniel Flower



In [62]:
Q = cosine_similarity(10, "Fantasy")

Cosine Similarity

Query: Fantasy

['fantasi']

[162 163 175 161 372 313  37 431 196  36]

The Blab Fantasy Novel

The Blab Fantasy novel-Ripped off and edited by The Slipped Disk

A Chance Meeting in the Park, by Joe DeRouen

Pricilla, the Trout Princess, by Gerald E. Gregg

John's Diner: Who Was The Guy? By Lucia Chambers

An Author Argues with his Main Character

Cliff-Hanger, a Poem

Valentine, Story by George Willard

The Game, by Suki

Angel's Fur, by Piper Sickles



In [65]:
Q = cosine_similarity(10, "engineer gadget")

Cosine Similarity

Query: engineer gadget

['engin', 'gadget']

[154  47   7   6  37   0   4 312 398 366]

The Adventure of the Engineer's Thumb

Collection of Additional Rules for Battletech, by Matt Murphy

What Research and Development Was Always Meant to Be, by M. Peshota

Engineering the Future of American Technology by M. Peshota (January 5, 1992)

Cliff-Hanger, a Poem

Going 100 West by 53 North by Jim Prentice (1990)

The Early Days of a High-Tech Start-up are Magic (November 18, 1991) by M. Peshota

Painkiller, by William Slattery

Startrek, The Ultimate Story!

Book Excerpt: Protoculture and the Children of the Shadow (Robotech)



In [64]:
Q = cosine_similarity(10, "spring closet")

Cosine Similarity

Query: spring closet

['spring', 'closet']

[251 138 139  41 211 410 400  60 292 123]

Keeping Insanity in the Closet, by The Weatherman (April 13th, 1988)

Disco Can By Fun, by Aldebaran of 11th Hour Productions

Disco Can Be Fun by Aldebaran of 11th Hour Productions

 A Collection of Various Good Stories

Nightmare in Grey by Fredric Brown

Tears of Glass by Kimberly Ann Carter (1992)

The Dark Sucker, by Quantum Mechanic

Super Blabnovel-A continuing saga on DATANET

Someone Talks About Some Weird Dreams He Has

Crazy Glue, a Novel by Jerry Slaff



In [59]:
def print_doc_custom(ids):
    for id in ids:
        tuple_ = dataset[id]
        location = tuple_[0]
        title = tuple_[1]
        print(title)
        #print(location)
        print("")

In [42]:
print_doc(200)

('/home/nutsa/Desktop/search_pdfs/stories/ghost', 'Time for Flowers, by Gay Bost')
TIME FOR FLOWERS
  by Gay Bost

They'd put flowers up. She hadn't noticed. Time wouldn't hold still.
She remembered, quite clearly, that time had been a simple thing; one
moment following the previous one, seconds strung out neatly like her
mother's pearls laid out on the dark mahogany vanity each Sunday
morning. But there had been a catch . . . 

Hung around Mother's neck the catch clicked and the tidy little line 
of seconds became a never ending circle with only the catch in the 
middle. For some reason the thought of pearls gathered from the sea, 
naturally nested within the confines of oyster shells, scattered 
haphazardly about the ocean floor disturbed her.

Now they'd put up the flowers in the same careless groupings. This,
too, disturbed her. Bright yellow trumpets, their collars spread to
catch the sun, dotted the front yard in clusters of two or three, five
or six. Bunches laid carelessly and 